# Predict regulatory regions from the DNA sequence with Janggu

In this notebook we illustrate several variants how to predict regulatory regions (of a toy example) from the DNA sequence.
The reference genome is made up of a concatenation of Oct4 and Mafk binding sites and we shall use all regions on chromosome 'pseudo1' as training
and 'pseudo2' as test chromosomes.

In [1]:
import os

import numpy as np
from keras import Model
from keras import backend as K
from keras.layers import Conv2D

from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.layers import Input
from keras.layers import Reshape

from pkg_resources import resource_filename

from janggu import Janggu
from janggu import Scorer
from janggu import inputlayer
from janggu import outputdense
from janggu.data import Bioseq
from janggu.data import Cover
from janggu.data import ReduceDim
from janggu.layers import DnaConv2D
from janggu.layers import LocalAveragePooling2D
from janggu.utils import ExportClustermap
from janggu.utils import ExportTsne
from janggu.utils import ExportTsv

from IPython.display import Image

np.random.seed(1234)

/home/wkopp/anaconda3/envs/jdev/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


First, we need to specify the output directory in which the results are stored.

In [2]:
os.environ['JANGGU_OUTPUT'] = '/home/wkopp/janggu_examples'

Specify the DNA sequence feature order. Order 1, 2 and 3 correspond to mono-, di- and tri-nucleotide based features (see Tutorial).

In [3]:
order = 3

In [4]:
# load the dataset
# The pseudo genome represents just a concatenation of all sequences
# in sample.fa and sample2.fa. Therefore, the results should be almost
# identically to the models obtained from classify_fasta.py.
REFGENOME = resource_filename('janggu', 'resources/pseudo_genome.fa')
# ROI contains regions spanning positive and negative examples
ROI_TRAIN_FILE = resource_filename('janggu', 'resources/roi_train.bed')
ROI_TEST_FILE = resource_filename('janggu', 'resources/roi_test.bed')
# PEAK_FILE only contains positive examples
PEAK_FILE = resource_filename('janggu', 'resources/scores.bed')

Load the datasets for training and testing

## Reusing the same dataset for training and test regions with view

When you have loaded the datasets using the store_whole_genome=True option, it is possible to reuse the same
dataset with different region of interests. To this end, the view method can be used to create another view on the dataset.
The advantage of this option is that the memory footprint will remain the same.

In [5]:
from janggu.data import view

In [6]:
DNA_TRAIN = Bioseq.create_from_refgenome('dna', refgenome=REFGENOME,
                                   roi=ROI_TRAIN_FILE,
                                   order=order,
                                   binsize=200,
                                   store_whole_genome=True)
                                   
LABELS_TRAIN = ReduceDim(Cover.create_from_bed('peaks', roi=ROI_TRAIN_FILE,
                               bedfiles=PEAK_FILE,
                               binsize=200,
                               resolution=200,
                               storage='sparse',
                               store_whole_genome=True))


DNA_TEST = view(DNA_TRAIN, ROI_TEST_FILE)
LABELS_TEST = view(LABELS_TRAIN, ROI_TEST_FILE)

In [7]:
DNA_TRAIN.shape, DNA_TEST.shape

((7797, 198, 1, 64), (200, 198, 1, 64))

In [8]:
LABELS_TRAIN.shape, LABELS_TEST.shape

((7797, 1), (200, 1))

## Define and fit a model

Neural networks can also be defined using the Janggu wrappers for keras.
This offers a few additional advantages, including reduced redundancy for defining models or automated evaluation.

First we define model using keras using a method. The decorators will automatically instantiate the initial layers and the output layers with the correct dimensionality. In the example above, this needed to be specified explicitly.

In [9]:
@inputlayer
@outputdense('sigmoid')
def double_stranded_model_dnaconv(inputs, inp, oup, params):
    """ keras model for scanning both DNA strands.

    A more elegant way of scanning both strands for motif occurrences
    is achieved by the DnaConv2D layer wrapper, which internally
    performs the convolution operation with the normal kernel weights
    and the reverse complemented weights.
    """
    with inputs.use('dna') as layer:
        # the name in inputs.use() should be the same as the dataset name.
        layer = DnaConv2D(Conv2D(params[0], (params[1], 1),
                                 activation=params[2]))(layer)
    output = GlobalAveragePooling2D(name='motif')(layer)
    return inputs, output


Now the model can be instantiated accordingly. We will also use a specific model name (which is optional).

In [10]:
modelname = 'dna2peak_ex4_order{}'.format(order)

In [11]:
# create a new model object
model = Janggu.create(template=double_stranded_model_dnaconv,
                      modelparams=(30, 21, 'relu'),
                      inputs=DNA_TRAIN,
                      outputs=LABELS_TRAIN,
                      name=modelname)

model.compile(optimizer='adadelta', loss='binary_crossentropy',
              metrics=['acc'])



Model fitting is similar as before.

Note also the use of ReduceDim which converts the original 4D Cover object to 2D table-like data structure. This is just for convenience, since it is also possible to set up a model as in the previous example.

In [ ]:
hist = model.fit(DNA_TRAIN, LABELS_TRAIN, epochs=100)

print('#' * 40)
print('loss: {}, acc: {}'.format(hist.history['loss'][-1],
                                 hist.history['acc'][-1]))
print('#' * 40)


Epoch 1/100
244/244 [==============================] - 4s 15ms/step - loss: 0.6372 - acc: 0.6297
Epoch 2/100
244/244 [==============================] - 3s 12ms/step - loss: 0.5187 - acc: 0.7661
Epoch 3/100
244/244 [==============================] - 3s 12ms/step - loss: 0.4618 - acc: 0.7984
Epoch 4/100
244/244 [==============================] - 3s 11ms/step - loss: 0.4225 - acc: 0.8230
Epoch 5/100
244/244 [==============================] - 3s 11ms/step - loss: 0.3930 - acc: 0.8318
Epoch 6/100
244/244 [==============================] - 3s 11ms/step - loss: 0.3680 - acc: 0.8473
Epoch 7/100
244/244 [==============================] - 3s 12ms/step - loss: 0.3450 - acc: 0.8548
Epoch 8/100
244/244 [==============================] - 3s 11ms/step - loss: 0.3237 - acc: 0.8672
Epoch 9/100
244/244 [==============================] - 3s 11ms/step - loss: 0.3023 - acc: 0.8795
Epoch 10/100
244/244 [==============================] - 3s 11ms/step - loss: 0.2814 - acc: 0.8922
Epoch 11/100
244/244 [=======

244/244 [==============================] - 3s 14ms/step - loss: 0.0189 - acc: 0.9981
Epoch 85/100
244/244 [==============================] - 3s 12ms/step - loss: 0.0188 - acc: 0.9978
Epoch 86/100
244/244 [==============================] - 3s 11ms/step - loss: 0.0183 - acc: 0.9983
Epoch 87/100
244/244 [==============================] - 3s 11ms/step - loss: 0.0177 - acc: 0.9986
Epoch 88/100
244/244 [==============================] - 3s 11ms/step - loss: 0.0171 - acc: 0.9983
Epoch 89/100
244/244 [==============================] - 3s 11ms/step - loss: 0.0167 - acc: 0.9988
Epoch 90/100
244/244 [==============================] - 3s 11ms/step - loss: 0.0164 - acc: 0.9986
Epoch 91/100
244/244 [==============================] - 3s 12ms/step - loss: 0.0156 - acc: 0.9990
Epoch 92/100
244/244 [==============================] - 3s 11ms/step - loss: 0.0155 - acc: 0.9992
Epoch 93/100
244/244 [==============================] - 3s 11ms/step - loss: 0.0151 - acc: 0.9988
Epoch 94/100
244/244 [===========

In [ ]:
# do the evaluation on the independent test data
model.evaluate(DNA_TEST, LABELS_TEST, datatags=['test'],
               callbacks=['auc', 'auprc', 'roc', 'prc'])

In [ ]:
evaluation_folder = os.path.join(os.environ['JANGGU_OUTPUT'], 'evaluation', modelname, 'test')

In [ ]:
Image(os.path.join(evaluation_folder, 'prc.png'))

In [ ]:
Image(os.path.join(evaluation_folder, 'roc.png'))